<a href="https://colab.research.google.com/github/henryonomakpo/The-Impact-of-ESG-Ratings-on-EV-Manufacturing-Industry/blob/main/E_commerce_ESG_Influence_on_Stock_Returns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# yfinance, statsmodels, pandas, numpy, scikit-learn, xlsxwriter, linearmodels
!pip install yesg
!pip install yfinance
!pip install statsmodels
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install xlsxwriter
!pip install linearmodels

  Preparing metadata (setup.py) ... done
  Created wheel for yesg: filename=yesg-2.1.1-py3-none-any.whl size=6105 sha256=21f8a2115b6675ee4e586f5938f7a9f0c29a3f38217086c3d29c78884a610785
  Stored in directory: /root/.cache/pip/wheels/78/8d/48/f5e8ff0315a46301e15c68371e297b460b33e1c846117725bc
Successfully built yesg
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.3 MB/s eta 0:00:00


### Fetch ESG dataset and save

In [9]:
# Required libraries: yesg, pandas
# Optional for Google Drive: google.colab
# !pip install yesg pandas

import yesg
import pandas as pd
import time # To add delays between API calls

# Attempt to import and use Google Drive specific libraries only if needed
try:
    from google.colab import drive
    google_colab_available = True
except ImportError:
    google_colab_available = False
    print("Google Colab environment not detected. Will save CSV locally.")

print("--- ESG Data Fetching Script for E-commerce Firms ---")

# --- Configuration ---

# List of tickers for E-commerce firms
TICKERS_ECOMMERCE = [
    'AMZN',   # Amazon
    'BABA',   # Alibaba (NYSE)
    'JD',     # JD.com
    'EBAY',   # eBay
    'WMT',    # Walmart
    'SE',     # Sea Limited (Shopee)
    'MELI',   # MercadoLibre
    'PDD',    # PDD Holdings
    'ETSY',   # Etsy
    'ZAL.DE', # Zalando
    'ALE.WA', # Allegro
    'TGT',    # Target
    '4755.T'  # Rakuten (Tokyo Stock Exchange)
]

# Define where to save the output file
DRIVE_MOUNT_PATH = '/content/drive'
OUTPUT_FILENAME = 'historic_esg_scores_ecommerce.csv'
OUTPUT_PATH_DRIVE = f'{DRIVE_MOUNT_PATH}/My Drive/{OUTPUT_FILENAME}' # Standard Google Drive path
OUTPUT_PATH_LOCAL = OUTPUT_FILENAME # Save in current directory if Drive fails

# Delay between API calls (in seconds) to avoid potential blocking
API_DELAY = 0.6

# --- Mount Google Drive (if in Colab) ---
drive_mounted = False
if google_colab_available:
    try:
        print(f"\nAttempting to mount Google Drive at {DRIVE_MOUNT_PATH}...")
        drive.mount(DRIVE_MOUNT_PATH)
        drive_mounted = True
        print("Google Drive mounted successfully.")
        save_path = OUTPUT_PATH_DRIVE
    except Exception as e:
        print(f"Failed to mount Google Drive: {e}")
        print(f"Output CSV will be saved locally as '{OUTPUT_PATH_LOCAL}'.")
        save_path = OUTPUT_PATH_LOCAL
else:
    # Not in Colab, saving locally
    save_path = OUTPUT_PATH_LOCAL

# --- Data Fetching Loop ---
print(f"\nTickers to fetch ESG data for: {TICKERS_ECOMMERCE}")
print("Starting ESG data download loop...")
print("WARNING: 'yesg' library relies on Yahoo Finance and may be outdated or have limited data coverage.")

# Initialize lists to store results and track progress
all_esg_data_list = []
successful_tickers = []
failed_tickers = []

for ticker in TICKERS_ECOMMERCE:
    print(f"  -> Processing: {ticker}")
    try:
        # Fetch all available historic ESG ratings for the ticker
        # Add the delay BEFORE the call
        time.sleep(API_DELAY)
        esg_scores_df = yesg.get_historic_esg(ticker)

        # Check if the result is a non-empty DataFrame
        if isinstance(esg_scores_df, pd.DataFrame) and not esg_scores_df.empty:
            # Add a column for the ticker symbol
            esg_scores_df['Ticker'] = ticker
            # Reset the index to make the date a column before appending
            esg_scores_df = esg_scores_df.reset_index()
            # Append the DataFrame to the list
            all_esg_data_list.append(esg_scores_df)
            successful_tickers.append(ticker)
            print(f"    -> Success: Found {len(esg_scores_df)} ESG data points for {ticker}.")
        else:
            # Handle cases where yesg returns None or an empty DataFrame
            print(f"    -> No valid ESG data found/returned for {ticker}")
            failed_tickers.append(ticker)
    except Exception as e:
        # Catch any other exceptions during fetching or processing
        print(f"    -> ERROR fetching/processing ESG data for {ticker}: {e}")
        failed_tickers.append(ticker)

# --- Combine and Save Data ---
if all_esg_data_list:
    print("\nCombining collected ESG data...")
    # Concatenate all the collected DataFrames into a single one
    final_esg_data = pd.concat(all_esg_data_list, ignore_index=True)

    # Standardize the date column name (it's often 'Date' or 'index' after reset_index)
    if 'index' in final_esg_data.columns and 'Date' not in final_esg_data.columns:
         final_esg_data = final_esg_data.rename(columns={'index': 'Date'})
    elif 'Date' not in final_esg_data.columns:
        print("Warning: Could not identify the primary date column after fetching. Please inspect the output.")

    # Attempt to convert Date column to datetime objects for consistency
    if 'Date' in final_esg_data.columns:
        try:
            final_esg_data['Date'] = pd.to_datetime(final_esg_data['Date'])
            print("  -> Date column converted to datetime.")
        except Exception as e:
            print(f"Warning: Could not convert 'Date' column to datetime format: {e}")

    # Display first few rows and info of the final DataFrame
    print("\nPreview of combined ESG data:")
    print(final_esg_data.head())
    print("\nData Info:")
    final_esg_data.info()

    # Save the combined data to the chosen CSV file path
    try:
        print(f"\nSaving ESG data to: {save_path} ...")
        final_esg_data.to_csv(save_path, index=False)
        print(f"ESG data saved successfully.")
    except Exception as e:
        print(f"\nERROR saving ESG data to CSV at '{save_path}': {e}")

else:
    # Message if no data was collected at all
    print("\nNo ESG data was successfully collected for any ticker. No CSV file created.")

# --- Final Summary ---
print("\n--- ESG Fetching Summary ---")
print(f"Successfully fetched ESG for ({len(successful_tickers)} tickers): {successful_tickers}")
print(f"Failed or no ESG data for ({len(failed_tickers)} tickers): {failed_tickers}")
print("--- Script Finished ---")

--- ESG Data Fetching Script for E-commerce Firms ---

Attempting to mount Google Drive at /content/drive...
Failed to mount Google Drive: Error: credential propagation was unsuccessful
Output CSV will be saved locally as 'historic_esg_scores_ecommerce.csv'.

Tickers to fetch ESG data for: ['AMZN', 'BABA', 'JD', 'EBAY', 'WMT', 'SE', 'MELI', 'PDD', 'ETSY', 'ZAL.DE', 'ALE.WA', 'TGT', '4755.T']
Starting ESG data download loop...
  -> Processing: AMZN
    -> Success: Found 128 ESG data points for AMZN.
  -> Processing: BABA
    -> Success: Found 128 ESG data points for BABA.
  -> Processing: JD
    -> Success: Found 98 ESG data points for JD.
  -> Processing: EBAY
    -> Success: Found 128 ESG data points for EBAY.
  -> Processing: WMT
    -> Success: Found 128 ESG data points for WMT.
  -> Processing: SE
    -> Success: Found 6 ESG data points for SE.
  -> Processing: MELI
    -> Success: Found 7 ESG data points for MELI.
  -> Processing: PDD
    -> Success: Found 7 ESG data points for PD

### Improved Model

### Robust Model
### Data Merging & Prep (Step 4): Carefully merges returns, factors, and loaded/lagged ESG data. It includes logic for forward-filling ESG scores before lagging and handles potential NaNs (either via imputation if IMPUTE_DATA=True or by dropping rows).

### VIF (Step 5): Checks multicollinearity on the final prepared panel data.

### *Panel Models (Step 6): Runs Pooled OLS, RE, FE (Entity), and FE (Two-Way) using appropriate formulas based on available variables after VIF checks.

### Specification Tests (Step 7): Performs Hausman and F-tests to compare models.



### Saving Results (Step 9): Saves all relevant outputs (panel summaries, tests, VIF, ML metrics, importance, predictions) to an Excel file.



In [9]:
# ==============================================================================
# --- Step 7: Specification Tests & Interpretation ---
# ==============================================================================
print("\n--- 7. Specification Tests & Interpretation (using Main Run results) ---")
spec_test_results_list = []
preferred_model_key = None # Reset

# Get results using the *new* keys for simple FE/RE models
fe_model = regression_results.get('FE_Entity_Simple') # Changed key
re_model = regression_results.get('RE_Simple')        # Changed key
pooled_model = regression_results.get('Pooled_Interaction') # Pooled still uses interaction
fe_tw_model = regression_results.get('FE_TwoWay_Simple')  # Changed key

is_fe_valid = isinstance(fe_model, PanelEffectsResults) # Check correct type
is_re_valid = isinstance(re_model, RandomEffectsResults)
is_pooled_valid = isinstance(pooled_model, PanelEffectsResults)
is_fe_tw_valid = isinstance(fe_tw_model, PanelEffectsResults)

# --- Run Specification Tests ---
try:
    # Hausman Test (FE vs RE) - Now comparing models with the *same* (simple) spec
    print("\n    Comparing FE (Simple) vs RE (Simple) - Hausman Test:")
    if is_fe_valid and is_re_valid:
        try:
            # Check common parameters (should match if using same formula)
            common_params = list(set(fe_model.params.index) & set(re_model.params.index))
            if not common_params:
                print("      -> Skipping Hausman: No common parameters.")
                spec_test_results_list.append({'Test': 'Hausman (FE vs RE - Simple)', 'Details': 'No common parameters', 'P-value': '-', 'Conclusion': 'Cannot Run'})
            else:
                print("      -> Performing Hausman test via model comparison...")
                comparison_fe_re = model_compare({"FE_Simple": fe_model, "RE_Simple": re_model})
                print(comparison_fe_re) # Print comparison table

                hausman_pval_str = "See Table"; pval_num = np.nan # Initialize
                # *** CORRECTED INDENTED TRY...EXCEPT BLOCK ***
                try: # Attempt to parse p-value from summary string
                    summary_str = str(comparison_fe_re)
                    match = re.search(r"Hausman\s+([\d\.]+)", summary_str)
                    if match: # This IF statement MUST be indented under the TRY
                        pval_num = float(match.group(1))
                        hausman_pval_str = f"{pval_num:.4f}"
                except Exception as parse_err:
                    print(f"       -> Warning: Could not parse Hausman p-value from table: {parse_err}")
                    pass # Ignore parsing errors, keep default "See Table"
                # *** END OF CORRECTED BLOCK ***

                # Determine conclusion based on parsed p-value if successful
                conclusion_hausman_test = 'Check Table' # Default
                if not pd.isna(pval_num):
                     conclusion_hausman_test = 'Prefer FE if Hausman p < 0.05'

                spec_test_results_list.append({
                    'Test': 'Hausman (FE vs RE - Simple)',
                    'Details': 'Comparison table printed',
                    'P-value': hausman_pval_str,
                    'Conclusion': conclusion_hausman_test
                })
        except Exception as comp_e:
            print(f"      -> Error running Hausman comparison: {comp_e}")
            spec_test_results_list.append({'Test': 'Hausman (FE vs RE - Simple)', 'Details': f"Error: {comp_e}", 'P-value': '-', 'Conclusion': 'Error'})
    else:
        details = "RE invalid" if is_fe_valid else "FE invalid" if is_re_valid else "Both invalid"
        print(f"\n    Skipping Hausman test: {details}.")
        spec_test_results_list.append({'Test': 'Hausman (FE vs RE - Simple)', 'Details': details, 'P-value': '-', 'Conclusion': 'Cannot Run'})


    # F-test for Poolability (Entity FE vs Pooled OLS) - Use FE_Simple vs Pooled_Interaction results
    print("\n    F-test for Poolability (Entity Effects):")
    if is_fe_valid: # Use the FE_Simple result
        try:
            if hasattr(fe_model, 'f_pooled'):
                f_pool = fe_model.f_pooled; stat_val = f_pool.stat; pval_val = f_pool.pval; df_num = getattr(f_pool, 'df_num', '?'); df_denom = getattr(f_pool, 'df_denom', '?')
                print(f"      F={stat_val:.4f}, P-value={pval_val:.4f} (df_num={df_num}, df_denom={df_denom})")
                conclusion = 'Reject Pooling (Use FE)' if pval_val < 0.05 else 'Cannot Reject Pooling (Pooled OK)'
                spec_test_results_list.append({'Test': 'F-test (Poolability - Entity)', 'Details': f'F({df_num},{df_denom})={stat_val:.4f}', 'P-value': f'{pval_val:.4f}', 'Conclusion': conclusion})
            else: print("      -> Poolability F-stat (f_pooled) not available."); spec_test_results_list.append({'Test': 'F-test (Poolability - Entity)', 'Details': 'f_pooled unavailable', 'P-value': '-', 'Conclusion': 'Cannot Run'})
        except Exception as ftest_e: print(f"      -> Error accessing Poolability F-test: {ftest_e}"); spec_test_results_list.append({'Test': 'F-test (Poolability - Entity)', 'Details': f"Error: {ftest_e}", 'P-value': '-', 'Conclusion': 'Error'})
    else: print("      -> Skipping Poolability F-test: FE (Simple) model invalid."); spec_test_results_list.append({'Test': 'F-test (Poolability - Entity)', 'Details': 'FE Invalid', 'P-value': '-', 'Conclusion': 'Cannot Run'})

    # F-test for Time Effects (Two-Way FE vs Entity FE) - Use FE_TwoWay_Simple vs FE_Entity_Simple
    print("\n    F-test for Time Effects:")
    if is_fe_tw_valid: # Use the FE_TwoWay_Simple result
         try:
            if hasattr(fe_tw_model, 'f_test_time'):
                f_time = fe_tw_model.f_test_time; stat_val = f_time.stat; pval_val = f_time.pval; df_num = getattr(f_time, 'df_num', '?'); df_denom = getattr(f_time, 'df_denom', '?')
                print(f"      F={stat_val:.4f}, P-value={pval_val:.4f} (df_num={df_num}, df_denom={df_denom})")
                conclusion = 'Time Effects Significant (Use Two-Way FE)' if pval_val < 0.05 else 'Time Effects Not Significant (Entity FE OK)'
                spec_test_results_list.append({'Test': 'F-test (Time Effects)', 'Details': f'F({df_num},{df_denom})={stat_val:.4f}', 'P-value': f'{pval_val:.4f}', 'Conclusion': conclusion})
            else: print("      -> Time Effects F-stat (f_test_time) not available."); spec_test_results_list.append({'Test': 'F-test (Time Effects)', 'Details': 'f_test_time unavailable', 'P-value': '-', 'Conclusion': 'Cannot Run'})
         except Exception as ftest_e: print(f"      -> Error accessing Time Effects F-test: {ftest_e}"); spec_test_results_list.append({'Test': 'F-test (Time Effects)', 'Details': f"Error: {ftest_e}", 'P-value': '-', 'Conclusion': 'Error'})
    else: print("      -> Skipping Time Effects F-test: Two-Way FE (Simple) model invalid."); spec_test_results_list.append({'Test': 'F-test (Time Effects)', 'Details': 'Two-Way FE Invalid', 'P-value': '-', 'Conclusion': 'Cannot Run'})

    spec_test_df = pd.DataFrame(spec_test_results_list)
except Exception as e: print(f"\nError during spec tests: {e}"); traceback.print_exc(); spec_test_df = pd.DataFrame()

# --- Determine Preferred Model (Revised Logic) ---
print("\n--- Preferred Model Selection Logic (Main Run) ---")
conclusion_pool = 'Cannot Run'; conclusion_time = 'Cannot Run'; conclusion_hausman = 'Cannot Run'
if not spec_test_df.empty:
    pool_row = spec_test_df[spec_test_df['Test'] == 'F-test (Poolability - Entity)']; conclusion_pool = pool_row['Conclusion'].iloc[0] if not pool_row.empty else 'Cannot Run'
    time_row = spec_test_df[spec_test_df['Test'] == 'F-test (Time Effects)']; conclusion_time = time_row['Conclusion'].iloc[0] if not time_row.empty else 'Cannot Run'
    hausman_row = spec_test_df[spec_test_df['Test'] == 'Hausman (FE vs RE - Simple)']; conclusion_hausman = hausman_row['Conclusion'].iloc[0] if not hausman_row.empty else 'Cannot Run'
print(f"  - Poolability Test Result: '{conclusion_pool}'"); print(f"  - Hausman Test (Simple Models) Result: '{conclusion_hausman}'"); print(f"  - Time Effects Test Result: '{conclusion_time}'")

preferred_model_key = None # Reset
# *** REVISED Model Selection Logic ***
if 'Time Effects Significant' in conclusion_time and is_fe_tw_valid:
    print("  - Logic: Time effects significant & Two-Way FE valid."); preferred_model_key = 'FE_TwoWay_Simple'; print("  -> Tentative Preference: FE Two-Way (Simple).")
elif 'Reject Pooling' in conclusion_pool and is_fe_valid:
    print("  - Logic: Pooling rejected & Entity FE valid.")
    if 'Prefer FE' in conclusion_hausman: print("  - Logic: Hausman also prefers FE."); preferred_model_key = 'FE_Entity_Simple'; print("  -> Tentative Preference: FE Entity (Simple).")
    else: print(f"  - Logic: Hausman ({conclusion_hausman}), but Poolability rejects Pooled. Prioritizing FE Entity."); preferred_model_key = 'FE_Entity_Simple'; print("  -> Tentative Preference: FE Entity (Simple).")
elif is_re_valid and 'Prefer FE' not in conclusion_hausman:
     print("  - Logic: FE not selected/valid, RE valid, Hausman doesn't strongly prefer FE."); preferred_model_key = 'RE_Simple'; print("  -> Tentative Preference: RE (Simple).")
elif 'Cannot Reject Pooling' in conclusion_pool and is_pooled_valid:
     print("  - Logic: FE/RE models not selected/valid, Pooling allowed, Pooled OLS valid."); preferred_model_key = 'Pooled_Interaction'; print("  -> Tentative Preference: Pooled OLS (Interaction).")
if preferred_model_key is None:
    print("\n  - No model selected by primary logic. Applying fallback...")
    fallback_order = ['FE_TwoWay_Simple', 'FE_Entity_Simple', 'RE_Simple', 'Pooled_Interaction']
    for model_key_fb in fallback_order:
        if model_key_fb in regression_results and isinstance(regression_results.get(model_key_fb), (PanelEffectsResults, RandomEffectsResults)):
            summary_val = model_summaries.get(model_key_fb)
            if not isinstance(summary_val, str): preferred_model_key = model_key_fb; print(f"  -> Fallback Selection: '{preferred_model_key}'."); break
            else: print(f"      -> Skipping fallback {model_key_fb} (summary failed).")
if preferred_model_key and (preferred_model_key not in regression_results or not isinstance(regression_results.get(preferred_model_key), (PanelEffectsResults, RandomEffectsResults))): print(f"  -> ERROR: Preferred model '{preferred_model_key}' not valid. Resetting."); preferred_model_key = None
if preferred_model_key and preferred_model_key in model_summaries and isinstance(model_summaries[preferred_model_key], str): print(f"  -> ERROR: Preferred model '{preferred_model_key}' failed summary. Resetting."); preferred_model_key = None
if preferred_model_key is None: print("\n  -> !!! CRITICAL: No valid model results available after fallback. !!!")
print(f"\n---> Final Preferred Model Selected (Main Run): {preferred_model_key if preferred_model_key else 'None (Review Logs)'} <---")

print(f"\n--- Interpretation (Based on '{preferred_model_key if preferred_model_key else 'Available Models'}') ---")
interpretation_provided = False
def get_coeff_info(results, var_name):
    if results and hasattr(results, 'params') and var_name in results.params.index:
        coeff = results.params.get(var_name); pval = results.pvalues.get(var_name); stderr = results.std_errors.get(var_name)
        sig_marker = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else '.' if pval < 0.1 else ''
        return f"Coeff={coeff:.4f}, SE={stderr:.4f}, Pval={pval:.4f} {sig_marker}"
    return f"Variable '{var_name}' not found or results invalid."

if preferred_model_key and preferred_model_key in regression_results and isinstance(regression_results[preferred_model_key], (PanelEffectsResults, RandomEffectsResults)):
    preferred_model_results = regression_results[preferred_model_key]; formula_used_key = model_formulas_used.get(preferred_model_key, "Unknown")
    print(f"    Interpreting Preferred Model: '{preferred_model_key}' (Formula Type: {formula_used_key})")
    if primary_esg_var:
        print(f"\n    Interpretation for '{primary_esg_var}':")
        if preferred_model_key == 'Pooled_Interaction' and any(':' in p for p in preferred_model_results.params.index):
             ref_cat_used = 'Unknown'; formula_string_to_parse = formula_pooled_interaction
             if formula_string_to_parse:
                  try: match = re.search(r"Treatment\(reference='([^']+)'\)", formula_string_to_parse); ref_cat_used = match.group(1) if match else 'Unknown'
                  except Exception: pass
             print(f"      Model includes interactions (Ref Cat: '{ref_cat_used}')"); print(f"      - Baseline Effect ({ref_cat_used}): {get_coeff_info(preferred_model_results, primary_esg_var)}"); print("      - (Interactions: See Pooled summary)")
        elif preferred_model_key in ['FE_Entity_Simple', 'FE_TwoWay_Simple', 'RE_Simple']:
             effects_description = "Unknown"
             if 'FE_Entity' in preferred_model_key: effects_description = "Entity Fixed"
             elif 'FE_TwoWay' in preferred_model_key: effects_description = "Entity & Time Fixed"
             elif 'RE' in preferred_model_key: effects_description = "Random Entity"
             print(f"      Model estimates avg effect controlling for {effects_description} effects."); print(f"      - Average Effect: {get_coeff_info(preferred_model_results, primary_esg_var)}")
        else: print(f"      - Unknown structure. Basic Effect: {get_coeff_info(preferred_model_results, primary_esg_var)}")
    else: print("    -> Primary ESG variable not specified/found.")
    print("\n    Interpretation Notes:"); print(f"      - Results based on '{preferred_model_key}'. Check Pval.");
    if IMPUTE_DATA and initial_missing_stats: print("      - !!! CAVEAT: Uses imputed data. High initial missingness. Compare w/ Sensitivity. !!!")
    print("      - Consider economic significance."); interpretation_provided = True
elif RUN_WITHOUT_IMPUTATION_SENSITIVITY:
    print("\n    -> Main run failed/not selected. Checking Sensitivity Run...")
    sens_key_to_interpret = None; sens_results = None; sens_pref_order = ['FE_Entity_Simple_Sens', 'Pooled_Interaction_Sens']
    for key in sens_pref_order:
        result_obj = sensitivity_regression_results.get(key) # Check result object
        summary_obj_or_err = sensitivity_summaries.get(key) # Check summary object/error
        if isinstance(result_obj, (PanelEffectsResults, RandomEffectsResults)) and not isinstance(summary_obj_or_err, str):
            sens_key_to_interpret = key; sens_results = result_obj; break
    if sens_key_to_interpret and sens_results:
        print(f"    -> Interpreting Sensitivity Model: '{sens_key_to_interpret}' (NO IMPUTATION) as fallback."); print("       !!! CAVEATS: Smaller subset of data. !!!")
        if primary_esg_var: print(f"       - ESG Effect ('{primary_esg_var}'): {get_coeff_info(sens_results, primary_esg_var)}")
        else: print("       - Primary ESG variable not found."); interpretation_provided = True
    else: print("    -> Sensitivity models also failed or were skipped.")
if not interpretation_provided: print("\n    -> No valid model results found to provide interpretation.")

# ==============================================================================
# --- Step 9: Print Consolidated Results to Console ---
# ==============================================================================
print("\n\n==============================================================================")
print(f"--- 9. Consolidated Analysis Results ({SCRIPT_VERSION}) ---")
print("==============================================================================")

print(f"\n--- Panel Model Summaries (Main Run - Imputed: {IMPUTE_DATA}) ---")
if not model_summaries: print("No models run/attempted.")
else:
    model_order = ['Pooled_Interaction', 'RE_Simple', 'FE_Entity_Simple', 'FE_TwoWay_Simple']
    for name in model_order:
        if name in model_summaries:
            result_or_error_summary = model_summaries[name]
            formula_used_key = model_formulas_used.get(name, "Unknown")
            print(f"\n--- Model: {name} ---"); print(f"    Formula Type Used: {formula_used_key}")
            if isinstance(result_or_error_summary, str): print(f"    -> Model Failed/Summary Error: {result_or_error_summary}")
            elif hasattr(result_or_error_summary, 'tables'):
                try:
                    if hasattr(result_or_error_summary, 'tables') and isinstance(result_or_error_summary.tables, list):
                         for table in result_or_error_summary.tables: print(table.as_text())
                    else: print(str(result_or_error_summary))
                except Exception as e: print(f"    -> Error formatting/printing summary tables for '{name}': {e}");
                try: print(str(result_or_error_summary))
                except: print("    -> Could not even convert summary object to string.")
            else: print(f"    -> Unknown result status stored (Type: {type(result_or_error_summary)}): {result_or_error_summary}")

if RUN_WITHOUT_IMPUTATION_SENSITIVITY:
    print("\n\n--- Panel Model Summaries (Sensitivity Run - NO IMPUTATION) ---")
    if not sensitivity_summaries: print("No sensitivity models run/results available.")
    else:
        sens_model_order = ['Pooled_Interaction_Sens', 'FE_Entity_Simple_Sens']
        for name in sens_model_order:
             if name in sensitivity_summaries:
                result_or_error_summary = sensitivity_summaries[name]
                formula_used_key = sensitivity_formulas_used.get(name, "Unknown")
                print(f"\n--- Model: {name} ---"); print(f"    Formula Type Used: {formula_used_key}")
                if isinstance(result_or_error_summary, str): print(f"    -> Model Failed/Summary Error: {result_or_error_summary}")
                elif hasattr(result_or_error_summary, 'tables'):
                     try:
                         if hasattr(result_or_error_summary, 'tables') and isinstance(result_or_error_summary.tables, list):
                              for table in result_or_error_summary.tables: print(table.as_text())
                         else: print(str(result_or_error_summary))
                     except Exception as e: print(f"    -> Error formatting/printing summary tables for '{name}': {e}");
                     try: print(str(result_or_error_summary))
                     except: print("    -> Could not even convert summary object to string.")
                else: print(f"    -> Unknown result status stored for {name} (Type: {type(result_or_error_summary)}): {result_or_error_summary}")

print(f"\n\n--- Preferred Model Selection & Comparison (Main Run) ---")
print(f"  -> Preferred model selected: {preferred_model_key if preferred_model_key else 'None (Review Logs)'}"); print(f"  -> Review specification tests and theory.")
if preferred_model_key and RUN_WITHOUT_IMPUTATION_SENSITIVITY:
    sens_key_map = {'Pooled_Interaction': 'Pooled_Interaction_Sens', 'FE_Entity_Simple': 'FE_Entity_Simple_Sens', 'FE_TwoWay_Simple': None, 'RE_Simple': None}
    sens_key = sens_key_map.get(preferred_model_key)
    if sens_key and sens_key in sensitivity_summaries:
        sens_result_or_error = sensitivity_summaries[sens_key]
        if not isinstance(sens_result_or_error, str): print(f"  -> COMPARISON: Sensitivity '{sens_key}' ran successfully. Compare results.")
        else: print(f"  -> COMPARISON NOTE: Sensitivity counterpart '{sens_key}' failed/skipped ({sens_result_or_error}).")
    elif sens_key: print(f"  -> COMPARISON NOTE: Sensitivity counterpart '{sens_key}' not found in sensitivity results.")
    else: print(f"  -> COMPARISON NOTE: No sensitivity counterpart defined for '{preferred_model_key}'.")

print("\n--- Interpretation Summary (Refer to Step 7) ---")
if interpretation_provided: print("  -> Interpretation provided in Step 7.")
else: print("  -> No interpretation provided (model failures).")

print("\n--- Specification Tests Summary (Main Run) ---")
if 'spec_test_df' in locals() and isinstance(spec_test_df, pd.DataFrame) and not spec_test_df.empty:
    try: print(spec_test_df.to_string(index=False, justify='left', max_colwidth=60))
    except Exception as print_err: print(f"  -> Print error: {print_err}"); print(spec_test_df)
else: print("  -> Specification tests unavailable.")

print("\n--- VIF Results (Main Run Data) ---"); print("  (VIF > 10 indicates potential issues)")
if vif_results_total is not None and isinstance(vif_results_total, pd.DataFrame): print("\n  VIF (Factors + Total ESG):"); print(vif_results_total.sort_values('VIF', ascending=False).to_string(index=False))
else: print("\n  VIF check (Total ESG) N/A or failed.")
if vif_results_components is not None and isinstance(vif_results_components, pd.DataFrame): print("\n  VIF (Factors + ESG Components):"); print(vif_results_components.sort_values('VIF', ascending=False).to_string(index=False))
else: print("\n  VIF check (Components) N/A or failed.")

print("\n\n--- Overall Reliability Assessment & Disclaimers ---")
print("  - Data Quality: Verify sources (Steps 2 & 3 logs).")
print(f"  - Imputation ({'MICE' if IMPUTE_DATA else 'Disabled'}): Main run {'used' if IMPUTE_DATA else 'did not use'} imputation.")
if IMPUTE_DATA and initial_missing_stats: high_missing_cols = [col for col, pct in initial_missing_stats.items() if pct > 25];
if IMPUTE_DATA and initial_missing_stats and high_missing_cols: print(f"    -> !!! CONCERN: High initial missingness (>25%): {high_missing_cols}.")
if IMPUTE_DATA: print("    -> Compare Main Run vs. Sensitivity Run (if successful).")
print("  - Model Specification & Validity:")
main_models_failed_count = sum(1 for res in model_summaries.values() if isinstance(res, str))
if main_models_failed_count == len(model_summaries): print("    -> !!! CRITICAL: All main run models failed. Review Step 6 logs. !!!")
elif main_models_failed_count > 0: print(f"    -> Warning: {main_models_failed_count} main run model(s) failed/summary error. Review logs.")
# Check specific model failures
if 'Pooled_Interaction' in model_summaries and isinstance(model_summaries['Pooled_Interaction'], str): print("    -> Warning: Pooled OLS model failed or had summary error.")
elif 'Pooled_Interaction' in regression_results and isinstance(regression_results['Pooled_Interaction'], PanelEffectsResults):
    try:
        pooled_f_robust = regression_results['Pooled_Interaction'].f_statistic_robust.stat
        # *** CORRECTED INDENTATION FOR TRY/EXCEPT BLOCK ***
        if not np.isfinite(pooled_f_robust) or pooled_f_robust < 0 :
             print("    -> Warning: Pooled OLS robust F-stat appears invalid. Check SE calculation.")
    except Exception: print("    -> Warning: Could not access Pooled OLS robust F-statistic."); pass
if 'RE_Simple' in model_summaries and isinstance(model_summaries['RE_Simple'], str): print("    -> Warning: RE model failed. Hausman test invalid.")
fe_models_to_check = ['FE_Entity_Simple', 'FE_TwoWay_Simple']
for fe_key in fe_models_to_check:
     if fe_key in model_summaries:
          summary_val = model_summaries[fe_key]
          if isinstance(summary_val, str) and ('Singular' in summary_val or 'Error' in summary_val): print(f"    -> Warning: {fe_key} failed summary ({summary_val[:60]}...).")
          elif not isinstance(summary_val, str) and hasattr(summary_val, 'summary'):
               summary_str = str(summary_val);
               if 'Absorbed' in summary_str or 'dropped' in summary_str.lower(): print(f"    -> Warning: {fe_key} summary indicates absorbed/dropped vars.")

print("    -> Note: FE/RE models used simplified spec (no category interactions). Pooled OLS used interactions.")
print("    -> Review model selection (Step 7) and theoretical fit.")
print("  - Data Characteristics: Check VIF results and Step 4 category warnings.")
print("\n  --- Conclusion ---"); print("  -> Treat findings with caution. Prioritize successful Sensitivity Run if Main had issues/imputation.")

print("\n==============================================================================")
print("--- End of Consolidated Results ---")
print("==============================================================================")

print(f"\n--- Script Finished ({SCRIPT_VERSION}) ---")


--- 7. Specification Tests & Interpretation (using Main Run results) ---

    Comparing FE (Simple) vs RE (Simple) - Hausman Test:
      -> Performing Hausman test via model comparison...
                      Model Comparison                     
                                FE_Simple         RE_Simple
-----------------------------------------------------------
Dep. Variable                ExcessReturn      ExcessReturn
Estimator                        PanelOLS     RandomEffects
No. Observations                      720               720
Cov. Est.                       Clustered         Clustered
R-squared                          0.1466            0.1445
R-Squared (Within)                 0.1466            0.1455
R-Squared (Between)               -1.7862           -0.2648
R-Squared (Overall)                0.1418            0.1445
F-statistic                        17.198            17.180
P-value (F-stat)                   0.0000            0.0000
======================     ====